# Text segmentation using Hidden Markov Models
> Tristan Perrot

## Automatic segmentation of mails, problem statement

- ***Q1:** Give the value of the π vector of the initial probabilities*

It is assumed that each mail actually contains a header : the decoding necessarily begins in the state 1. Then the $\pi$ vector is defined as :
$$
\pi = \begin{pmatrix} 1 \\ 0 \end{pmatrix}
$$

- ***Q2:** What is the probability to move from state 1 to state 2 ? What is the probability to remain in state 2 ? What is the lower/higher probability ? Try to explain why*

The transition matrix estimated on a labeled small corpus has the following form :
$$
A = \begin{pmatrix} 0.999218078035812 & 0.000781921964187974 0 \\ 0 & 1 \end{pmatrix}
$$

The probability to move from state 1 to state 2 is $0.000781921964187974$, and the probability to remain in state 2 is $1$. The **lower** probability is the probability to **move from state 1 to state 2** and the **higher** is to **remain in the state 2**. This is due to the fact that since we moved from the header to the body it's impossible to see again the body because we know that each mail contains exactly one header and one body, each mail follows once the transition from 1 to 2.

- ***Q3:** What is the size of B ?*

$B$ is the observation matrix. $N$ is the number of different characters. Since each part of the mail is characterized by a discrete probability distribution on the characters $P(c|s)$, with $s = 1$ or $s = 2$. Then, the shape of $B$ is $(N, 2)$.

## Material

### Coding/decoding mails

In [34]:
import numpy as np
import os
import glob

In [35]:
ROOT = os.path.abspath('.')

PERL_DIR = os.path.join(ROOT,'PerlScriptAndModel')
OBS_PATH = os.path.join(PERL_DIR, 'P.text')

OUTPUT_DIR = os.path.join(ROOT,'out')

DATA_DIR = os.path.join(ROOT,'dat')
MAIL_LIST_PATH = os.path.join(DATA_DIR, 'mail.lst')

In [36]:
# Load the list of files

# Iterate through files and load the text 
def files_iter(data_dir, with_name=False):
    files = glob.glob('{}/*.dat'.format((data_dir)))
    if with_name:
        for f in files:
            # Get the filename
            if '/' in f:
                name = f.split('/')[-1].split('.')[0]
            else:
                name = f.split('\\')[-1].split('.')[0]
            # Return filename and associated text
            yield name, np.loadtxt(f, dtype=int)
    else:
        for f in files:
            # Return text
            yield np.loadtxt(f, dtype=int)

# Config
pi = np.array([1, 0])
A = np.array([[0.999218078035812, 0.000781921964187974], [0, 1]])
B = np.loadtxt(OBS_PATH)

In [37]:
def viterbi(observations, pi, A, B):
    # Number of states and observations
    num_states = A.shape[0]
    num_observations = len(observations)

    logA = np.log(A + np.finfo(np.float64).tiny)
    logB = np.log(B + np.finfo(np.float64).tiny)
    log_pi = np.log(pi + np.finfo(np.float64).tiny)

    # Initialize the Viterbi trellis and backpointers
    viterbi_trellis = np.full((num_states, num_observations), -np.inf)
    backpointers = np.zeros((num_states, num_observations), dtype=int)

    # Initialize the first column of the Viterbi trellis
    viterbi_trellis[:, 0] = log_pi + logB[observations[0], :]

    # Perform the Viterbi algorithm
    for t in range(1, num_observations):
        for s in range(num_states):
            # Calculate the maximum probability and corresponding backpointer
            max_prob = np.max(viterbi_trellis[:, t-1] + logA[:, s] + logB[observations[t], s])
            backpointers[s, t] = np.argmax(viterbi_trellis[:, t-1] + logA[:, s] + logB[observations[t], s])

            # Update the Viterbi trellis with the maximum probability
            viterbi_trellis[s, t] = max_prob

    # Find the sequence of states with the highest probability
    best_sequence = [np.argmax(viterbi_trellis[:, -1])]
    for t in range(num_observations-1, 0, -1):
        best_sequence.append(backpointers[best_sequence[-1], t])

    best_sequence.reverse()
    return best_sequence

In [38]:
mails = [mail for mail in files_iter(DATA_DIR)]

# Test Viterbi on some mails that are given in the dat directory (especially mail11.txt to mail30.txt).
print("Mail 11 :")
print(viterbi(mails[11], pi, A, B))

print("Mail 30 :")
print(viterbi(mails[29], pi, A, B))

Mail 11 :
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [39]:
# Creating a directory to put the result of the viterbi function
if not os.path.exists(OUTPUT_DIR):
    os.mkdir(OUTPUT_DIR)

# Function that will write a viterbi path for a mail in a dedicated result file
def create_viterbi_path_file(mail_name, viterbi_path):
    with open('{}/{}_segmented.txt'.format(OUTPUT_DIR, mail_name), 'w') as f: 
        f.write(''.join([str(c) for c in viterbi_path]))   

In [40]:
# Creating a result file for each mail
mail_iter = files_iter(DATA_DIR, with_name=True)

# Using our generator, we get the mail names and data
for name_file, data in mail_iter:
    # Find out the viterbi path using viterbi
    viterbi_path = viterbi(data, pi, A, B)
    # Put it in the result file
    create_viterbi_path_file(name_file, viterbi_path)

### Visualizing segmentation

In [43]:
# Writing a function to go into the directory and execute the perl script "segment.pl" on the mail in the given path
def exec_perl_script(mail, path):
    return os.system('perl {}/segment.pl {} {}'.format(PERL_DIR, mail, path))

# Writing a function getting the original mail, the result of viterbi, and applying the segmentation script
# Then putting the result
def segment_mail(mail_name, data_dir, output_dir):
    # Get the full path of the mail
    mail = os.path.join(data_dir, '{}.txt'.format(mail_name))
    # Get the full path of the result
    path = os.path.join(output_dir, '{}_segmented.txt'.format(mail_name))
    # Execute the segmentation script
    exec_perl_script(mail, path)
    # Read the segmented mail
    with open(path, 'r') as f:
        segmented_mail = f.read()
    # Split the segmented mail into header and body based on the cutting line
    cutting_line = '---CUT---'
    if cutting_line in segmented_mail:
        header, body = segmented_mail.split(cutting_line, 1)
        return header, body
    else:
        return segmented_mail, None

In [44]:
# Getting mails names
mail_names = [name for name, _ in files_iter(DATA_DIR, with_name=True)]

# Call the function and look at the result of segmentation
for mail_name in mail_names:
    header, body = segment_mail(mail_name, DATA_DIR, OUTPUT_DIR)
    print('Mail:', mail_name)
    print('Header:', header)
    print('Body:', body)
    print('---')

1
Mail: mail1
Header: 000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

- ***Q4:** Print the track and present and discuss the results obtained on mail11.txt to mail30.txt*

- ***Q5:** How would you model the problem if you had to segment the mails in more than two parts (for example : header, body, signature) ?*

In this case, the model will have more states, if we denote $Q$ the number of states then the shape of $A$ will be $(Q,Q)$, the shape of $B$ will be $(N,Q)$ and $\pi$ will be equal to $\begin{pmatrix} 1 \\ 0 \\ 0 \end{pmatrix}$.The rest will be the same.

- ***Q6:** How would you model the problem of separating the portions of mail included, knowing that they always start with the character ">"*
